In [9]:
# importamos las librerias necesarioas, para crear nuewtra red neuronal, transformar las imagenes a una matriz, y descargar el dataset
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [2]:

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

100%|███████████████████████| 170498071/170498071 [00:10<00:00, 17017885.26it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [4]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [7]:
class RedClasificadora(nn.Module):
    def __init__(self):
        super(RedClasificadora, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 512) 
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self,x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8) 
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.softmax(x)
        return x
        
    

In [10]:

model = RedClasificadora()

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [11]:

model.train()

num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99: 
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

Epoch [1/10], Step [100], Loss: 2.3027
Epoch [1/10], Step [200], Loss: 2.3024
Epoch [1/10], Step [300], Loss: 2.3021
Epoch [1/10], Step [400], Loss: 2.3018
Epoch [1/10], Step [500], Loss: 2.3012
Epoch [1/10], Step [600], Loss: 2.3007
Epoch [1/10], Step [700], Loss: 2.3002
Epoch [1/10], Step [800], Loss: 2.3001
Epoch [1/10], Step [900], Loss: 2.2997
Epoch [1/10], Step [1000], Loss: 2.2989
Epoch [1/10], Step [1100], Loss: 2.2980
Epoch [1/10], Step [1200], Loss: 2.2994
Epoch [1/10], Step [1300], Loss: 2.2979
Epoch [1/10], Step [1400], Loss: 2.2967
Epoch [1/10], Step [1500], Loss: 2.2957
Epoch [1/10], Step [1600], Loss: 2.2950
Epoch [1/10], Step [1700], Loss: 2.2935
Epoch [1/10], Step [1800], Loss: 2.2900
Epoch [1/10], Step [1900], Loss: 2.2850
Epoch [1/10], Step [2000], Loss: 2.2802
Epoch [1/10], Step [2100], Loss: 2.2805
Epoch [1/10], Step [2200], Loss: 2.2798
Epoch [1/10], Step [2300], Loss: 2.2624
Epoch [1/10], Step [2400], Loss: 2.2454
Epoch [1/10], Step [2500], Loss: 2.2591
Epoch [1/

In [12]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified


In [14]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images,labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')
        


Accuracy of the network on the 10000 test images: 66.64%


In [15]:
from PIL import Image


In [30]:
import requests
from io import BytesIO
from PIL import Image
import torchvision.transforms as transforms
import torch

def load_image(image_path):
    if image_path.startswith('http://') or image_path.startswith('https://'):
        response = requests.get(image_path)
        image = Image.open(BytesIO(response.content))
    else:
        image = Image.open(image_path)

    transform = transforms.Compose(
        [transforms.Resize((32, 32)),
         transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    image = transform(image)
    image = image.unsqueeze(0)
    return image

def predict(image_path):
    image = load_image(image_path)
    with torch.no_grad():
        out = model(image)
        _, predicted = torch.max(out.data, 1)
    return predicted.item()


image_path = 'https://www.princeton.edu/sites/default/files/styles/half_16_9_1440/public/images/2022/02/KOA_Nassau_2697x1517.jpg?itok=Nd5wwOFu'
predicted_class = predict(image_path)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
print(f'clase predicha: {classes[predicted_class]}')

The predicted class is: dog
